# Disaster Tweets Prediction

## Preparation

Import libraries

In [33]:
import pandas as pd
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

In [2]:
training_set = pd.read_csv("datasets/train.csv")
testing_set = pd.read_csv("datasets/test.csv")

In [3]:
training_set.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
training_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [10]:
training_set.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [13]:
class_dist = training_set.groupby("target").size()

for index, val in class_dist.iteritems():
    percentage = (val / sum(class_dist) * 100)
    print(f"Class {index} : {val} samples ({percentage:.2f}%)")

Class 0 : 4342 samples (57.03%)
Class 1 : 3271 samples (42.97%)


In [16]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
    return text

In [19]:
training_set["text"] = training_set["text"].apply(preprocessor)

In [20]:
training_set.head()

,id,keyword,location,text,target
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,all residents asked to shelter in place are be...,1
3,6,NaN,NaN,13 000 people receive wildfires evacuation ord...,1
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1


In [28]:
X_train = training_set["text"]
y_train = training_set["target"]

In [29]:
tfidf_vectorizer = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)
clf_method = LogisticRegression()

clf = Pipeline([
    ('vectorizer', tfidf_vectorizer),
    ('classifier', clf_method)
])

In [30]:
clf.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer(lowercase=False)),
                ('classifier', LogisticRegression())])

In [31]:
training_predicted = clf.predict(X_train)

In [34]:
training_accuracy = accuracy_score(y_train, training_predicted)
print(f'Accuracy on training set: {training_accuracy:.3f}')

Accuracy on training set: 0.884
